<div style='background-image: url("baku.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.9) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">8th Munich Earth Skience School</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Sudelfeld, 4-9 March 2018</div>
        </div>
    </div>


<p style="width:50%;float:right;padding-left:50px">
<img src=./mess.jpg>
<span style="font-size:smaller">
</span>
</p>

# 6-C polarization analysis using point measurements of translational and rotational ground-motion

## Exercise 1 isolated P-wave 

### Authors:
* David Sollberger (ETH Zurich)
* Cedric Schmelzbach (ETH Zurich)
* Cederic van Renterghem (ETH Zurich)
* Taufiqurrahman (LMU Munich)

***

In this notebook we will show an example of the application of the proposed single-station 6-C MUSIC algorithm to a simple synthetic data set. We will analyze a P-wave, recorded at the free surface, with polarization parameters $m^P = (θ_P = 20◦, φ = 20◦$, $α = 300 ms −1$ , $β =173 ms −1 )^T$ . Synthetic data were created by convolving spikes of appropriate relative amplitudes as described by the polarization model in eq. above with a band-limited Ricker-wavelet (5 Hz centre-frequency). The synthetic data were additionally contaminated with random noise with equal power on all six components. The MUSIC likelihood is colour-coded and displayed on a logarithmic scale in order to better show the global topography of the function. Note that the MUSIC function has a single, distinct maximum where the tested model parameters match the true parameters (displayed with a dot).

At the free surface, the pure-mode polarization vectors v for a P-wave become (see Sollberger et.al.(2018) for details),

$$
{v_P^{FS}} = 
\begin{pmatrix} 
-p_s[sin(\theta_p)cos(\phi)+\frac{A_{PP}}{A_P}sin(\theta_p)cos(\phi)+\frac{A_{PS}}{A_P}cos(\theta_s)cos(\phi)] \\
-p_s[sin(\theta_p)sin(\phi)+\frac{A_{PP}}{A_P}sin(\theta_p)sin(\phi)+\frac{A_{PS}}{A_P}cos(\theta_s)sin(\phi)] \\
-p_s[cos(\theta_p)-\frac{A_{PP}}{A_P}cos(\theta_p)+\frac{A_{PS}}{A_P}sin(\theta_s)] \\
(2\beta)_{-1}\frac{A_{PS}}{A_P}sin(\phi) \\
-(2\beta)_{-1}\frac{A_{PS}}{A_P}cos(\phi) \\
0
\end{pmatrix}
$$

From equation above, it is apparent that the free surface 6-C polarization vectors for a P-wave, respectively, can be fully described by the polarization parameter vectors: $m^P = (\theta_P, \phi, \alpha, \beta)^T$ Note that $\beta$ is explicit in equation but $\alpha$ is implicit through the angles $\theta_S$ and $\theta_P$ from Snell’s law relation $(\frac{sin \theta_P}{\alpha} = \frac{sin \theta_S}{\beta})$. We now seek to find the parameter vector $m$, that best describes the polarization vector for a specified wave type in an analysis window, even if there is interference of other waves.

For a given wave with wavefield parameter vector $m$, we define a likelihood function $L(d|m)$ : $\mathbb{R}^n$ → $\mathbb{R}^1$ (where $n$ is the number of elements in $m$) that maximizes when the parameters of m best describe the measured data $d$. The best-fitting parameter vector $m̂$ can then be found by a grid search through the solution space of $L(d|m)$:
$$
m̂ = arg max \ L(d|m)
$$

In order to define $L(d|m)$, we adapt the multiple signal classification algorithm (MUSIC) that was first introduced by Schmidt (1986). MUSIC is a very high-resolution signal detection approach and is widely used in radio direction finding and sonar. The algorithm explores the null-space of the coherency matrix C and yields accurate estimates of the model parameters. A major advantage of the MUSIC algorithm, is that it enables the detection of multiple interfering signals.

First, we perform an eigendecomposition of the 6 × 6 coherency matrix $C$. The six eigenvectors e n (for $n = 1, ..., 6$) of $C$ are then sorted in descending order of their eigenvalues such that $e_1$ is the eigenvector associated with the largest eigenvalue and $e_6$ the one associated with the smallest eigenvalue. We now define the null-space of the coherency matrix by taking the outer product of the minimum eigenvectors, yielding the following projection matrix:

$$Q(d) = (e_{6−l} · · · e_5 e_6 )(e_{6−l} · · · e_5 e_6 )^T$$

where $l$ is chosen with respect to the number of linearly polarized events that are to be resolved. We now define the MUSIC likelihood function as (Schmidt 1986):

$$
L(d|m) = \frac{1}{v̂^T(m)Q(d)v̂(m)}
$$

where $v̂(m)$ describes the 6-C polarization of a pure-state wave arrival of a specified mode of vibration as given by the polarization models in previous equations, normalized to a unit vector $(v̂ = \frac{v}{||v||})$. Due to the nonlinearity of the function $L(d|m)$, it is very sensitive to the global maximum.

In [ ]:
# import libraries 
# ----------------
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt

from ricker     import *
from awgn       import *
from polvect6C  import *
from music6C    import *

class structtype():
    pass

In [ ]:
# MUSIC parameters 
# ----------------
test_param       = structtype()
test_param.vp    = np.arange(200,500+25,25) # parameter space for vp
test_param.vs    = np.arange(100,290+5,5)   # parameter space for vs
test_param.theta = np.arange(0,90+5,5)      # parameter space for theta
test_param.phi   = np.arange(-180,180+5,5)  # parameter space for phi
wl = 0.4                                    # window length in seconds

# wave parameters
theta  = 20.                      # incidence angle [degree]
phi    = 20.                      # azimuth [degree]
vp     = 300.                     # P-wave velocity at the receiver [m/s]
vs     = 170.                     # S-wave velocity at the receiver [m/s]
v_scal = 2 * vs                   # scaling velocity (slightly overestimated S-wave velocity)

# seismogram parameters
tmin = 0.                         # minimum time of the seismogram [s]
tmax = 1.5                        # maximum time of the seismogram [s]
dt   = 0.25e-3                    # sampling interval [s]
t    = np.arange(tmin,tmax+dt,dt) # time [s]
t    = np.round(t,6)
t_1  = 0.75                       # arrival time of the wave [s]
fc   = 5.                         # dominant frequency [Hz]
wav  = ricker(dt,fc,tmax)         # Ricker wavelet

# generate synthetics
param       = structtype()
param.theta = theta
param.phi   = phi
param.vp    = vp
param.vs    = vs

In [ ]:
# 6-C polarization vector 
# -----------------------
v = polvect6C(param,v_scal,'P')   # calculate the polarization vector for the given wave
v = v / np.sqrt(sum(v**2))        # convert to unit vector

data = (np.zeros((len(t),6)))     # zero data initialization
indx = t.tolist().index(t_1)      # index of arrival time of the wave 
data[indx,:] = v

for i in range(0, data[0,:].size - 1):
    data[:,i] = np.convolve(data[:,i], wav, 'same')
    
# add random noise 
SNR  = 50.                        # signal to noise ratio
data = awgn(data,SNR)

In [ ]:
# 6-C MUSIC 
# ---------
W = (wl / dt) # window length in samples
L = music6C(data,test_param,'P',v_scal,indx,W,'auto',0,0.01)

In [ ]:
# remove instabilities 
# --------------------
L_ = np.nan_to_num(L)
L_[L_ == 0.] = 1.

# determine maximum value and index of the MUSIC estimator 
maxval  =    np.max(abs(L_[:]))
maxidx  = np.argmax(abs(L_[:]))
i,j,k,l = np.unravel_index(maxidx, L_[:].shape) # column- to row-major indexing

phitheta  = np.squeeze((abs(L_[:,:,k,l]))) # plane through best-fitting velocity parameters
vpvs      = np.squeeze((abs(L_[i,j,:,:]))) # plane through best-fitting arrival angles

In [ ]:
# plot configuration 
# ------------------
plt.figure(figsize=(8, 4))

plt.subplot(1,2,1)
vm1 = np.percentile((abs(phitheta)), 100)
plt.imshow((abs(phitheta)), cmap="jet", vmin=np.min((abs(phitheta))), vmax=vm1, aspect='auto', extent=[test_param.phi[0],
                                                                                                       test_param.phi[-1], 
                                                                                                       test_param.theta[-1], 
                                                                                                       test_param.theta[0]])
plt.xlabel(r'$\phi$ (°)')
plt.ylabel(r'$\theta$ (°)')
plt.plot(phi,theta,'r.')

plt.subplot(1,2,2)
vm2 = np.percentile((abs(vpvs)), 100)
plt.imshow(abs(vpvs), cmap="jet", vmin=np.min((abs(vpvs))), vmax=vm2, aspect='auto', extent=[test_param.vs[0],
                                                                                             test_param.vs[-1], 
                                                                                             test_param.vp[-1], 
                                                                                             test_param.vp[0]])
plt.xlabel(r'$v_S$ (m/s)')
plt.ylabel(r'$v_P$ (m/s)')
plt.plot(vs,vp,'ro')

# show result on screen
plt.tight_layout()
plt.show()